In [4]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib

import mne
import pandas as pd
from eztrack import (
    read_derivative
)
from eztrack.base.statistics.sampling import resample_seizure
from eztrack.base.utils.file_utils import _replace_ext
from eztrack.io import read_result_eztrack, read_clinical_excel
from eztrack.io.base import _add_desc_to_bids_fname, DERIVATIVETYPES
from eztrack.viz.plot_fragility_ts import plot_fragility_ts
from mne.viz import plot_alignment, snapshot_brain_montage
from mne_bids import BIDSPath, get_entity_vals
from mne_bids.path import get_entities_from_fname, _parse_ext
import matplotlib.animation as animation

Using matplotlib backend: TkAgg


In [5]:
from matplotlib import animation, rc
from IPython.display import HTML

In [6]:
def run_plot_heatmap(deriv_path, figures_path):
    deriv = read_derivative(deriv_path)

    # read in channel types
    bids_path = BIDSPath(**deriv.info.source_entities,
                         datatype=deriv.info['datatype'],
                         root=deriv.bids_root)
    bids_path.update(suffix='channels', extension='.tsv')

    # read in sidecar channels.tsv
    channels_pd = pd.read_csv(bids_path.fpath, sep='\t')
    description_chs = pd.Series(channels_pd.description.values, index=channels_pd.name).to_dict()
#     print(description_chs)
    resected_chs = [ch for ch, description in description_chs.items() if description == 'resected']
#     print(f'Resected channels are {resected_chs}')

    # set title name as the filename
    title = Path(deriv_path).stem

    figure_fpath = Path(figures_path) / Path(deriv_path).with_suffix('.pdf').name

    # normalize
    deriv.normalize()
    cbarlabel = 'Fragility'

    subject = 'fsaverage'
    subjects_dir = '/Users/adam2392/Dropbox/epilepsy_bids/derivatives/freesurfer'
    electrodes_fpath = '/Users/adam2392/Downloads/nihcoords/NIH040.csv'
    mnicoords_fpath = '/Users/adam2392/Downloads/nihcoords/pt1.txt'
    elec_df = pd.read_csv(electrodes_fpath)
    ch_names = elec_df['chanName'].tolist()
    ch_coords = elec_df[['x', 'y', 'z']].to_numpy(dtype=float) / 1000.
    ch_coords = np.loadtxt(mnicoords_fpath, dtype=float, delimiter='\t') / 1000.
    ch_pos = dict(zip(ch_names, ch_coords))
    print(ch_pos)

    lpa, nasion, rpa = mne.coreg.get_mni_fiducials(
        subject, subjects_dir=subjects_dir)
    lpa, nasion, rpa = lpa['r'], nasion['r'], rpa['r']

    montage = mne.channels.make_dig_montage(
        ch_pos, coord_frame='mri', nasion=nasion, lpa=lpa, rpa=rpa)
    print('Created %s channel positions' % len(ch_names))

    trans = mne.channels.compute_native_head_t(montage)
    print(trans)

    info = mne.create_info(ch_names=deriv.ch_names, sfreq=8.0)
    raw = mne.io.RawArray(deriv.get_data(), info)
    # attach montage
    raw.set_montage(montage)

    fig = plot_alignment(raw.info, subject=subject, subjects_dir=subjects_dir,
                         surfaces=['pial'], trans=trans, coord_frame='mri')
    mne.viz.set_3d_view(fig, 200, 70, focalpoint=[0, -0.005, 0.03])

    xy, im = snapshot_brain_montage(fig, montage)

    # Convert from a dictionary to array to plot
    xy_pts = np.vstack([xy[ch] for ch in raw.info['ch_names']])

    # create an initialization and animation function
    # to pass to FuncAnimation
    def init():
        """Create an empty frame."""
        return paths,

    def animate(i, activity):
        """Animate the plot."""
        paths.set_array(activity[:, i])
        return paths,
    cmap = 'turbo'
    vmin = 0
    vmax = 0.7
    # create the figure and apply the animation of the
    # gamma frequency band activity
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(im)
    ax.set_axis_off()
    paths = ax.scatter(*xy_pts.T, c=np.zeros(len(xy_pts)), s=200,
                       cmap=cmap, vmin=vmin, vmax=vmax)
    fig.colorbar(paths, ax=ax)
    ax.set_title('Gamma frequency over time (Hilbert transform)',
                 size='large')

    # avoid edge artifacts and decimate, showing just a short chunk
    show_power = raw.get_data()[:, 100:700:2]
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   fargs=(show_power,),
                                   frames=show_power.shape[1],
                                   interval=100, blit=True)
    
    # equivalent to rcParams['animation.html'] = 'html5'
    rc('animation', html='html5')
    print(anim)
    print(show_power.shape)
#     plt.show(block=True)
#     plt.waitforbuttonpress()
    # plt.pause(0.001)
    # raise Exception('hi')
    # run heatmap plot
    # deriv.plot_heatmap(
    #     cmap='turbo',
    #     cbarlabel=cbarlabel,
    #     title=title,
    #     figure_fpath=figure_fpath,
    #     soz_chs=resected_chs,
    # )
    print(f'Saved figure to {figure_fpath}')


In [7]:
# the root of the BIDS dataset
WORKSTATION = "home"

if WORKSTATION == "home":
    # bids root to write BIDS data to
    root = Path('/Users/adam2392/Dropbox/epilepsy_bids/')
    source_dir = root / 'sourcedata'
    deriv_root = root / 'derivatives' / 'interictal'

    figures_root = deriv_root / 'figures'

    # path to excel layout file - would be changed to the datasheet locally
    excel_fpath = source_dir / 'organized_clinical_datasheet_raw.xlsx'
elif WORKSTATION == "lab":
    bids_root = Path("/home/adam2392/hdd2/epilepsy_bids/")
    source_dir = Path("/home/adam2392/hdd2/epilepsy_bids/sourcedata")
    excel_fpath = Path(
        "/home/adam2392/hdd2/epilepsy_bids/sourcedata/organized_clinical_datasheet_raw.xlsx"
    )

    # output directory
    deriv_path = Path("/home/adam2392/hdd/openneuro/derivatives")

    # figures directory
    figures_dir = Path(
        "/home/adam2392/hdd2/epilepsy_bids/derivatives/fragility/figures"
    )

# define BIDS entities
SUBJECTS = [
    'pt1'
]

# pre, Sz, Extraoperative, post
task = "interictal"
acquisition = "ecog"
datatype = "ieeg"
extension = ".vhdr"
session = "presurgery"  # only one session

# analysis parameters
reference = 'average'
sfreq = None

# get the runs for this subject
all_subjects = get_entity_vals(root, "subject")
for subject in all_subjects:
    if subject not in SUBJECTS:
        continue
    ignore_subs = [sub for sub in all_subjects if sub != subject]

    # get all sessions
    ignore_set = {
        'ignore_subjects': ignore_subs,
    }
    print(f'Ignoring these sets: {ignore_set}')
    all_tasks = get_entity_vals(root, "task", **ignore_set)
    tasks = all_tasks
    tasks = ['interictal']

    for task in tasks:
        print(f"Analyzing {task} task.")
        ignore_tasks = [tsk for tsk in all_tasks if tsk != task]
        ignore_set['ignore_tasks'] = ignore_tasks
        runs = get_entity_vals(
            root, 'run', **ignore_set
        )
        print(f'Found {runs} runs for {task} task.')

        for idx, run in enumerate(runs):
            # create path for the dataset
            bids_path = BIDSPath(
                subject=subject,
                session=session,
                task=task,
                run=run,
                datatype=datatype,
                acquisition=acquisition,
                suffix=datatype,
                root=root,
                extension=extension,
            )
            print(f"Analyzing {bids_path}")

            deriv_basename = _add_desc_to_bids_fname(bids_path.basename,
                                                     description=DERIVATIVETYPES.COLPERTURB_MATRIX.value)
            deriv_chain = Path('originalsampling') / "fragility" / reference / f"sub-{subject}"
            deriv_path = deriv_root / deriv_chain / deriv_basename
            figures_path = figures_root / deriv_chain
            run_plot_heatmap(deriv_path=deriv_path, figures_path=figures_path)
            
            break
        break

Ignoring these sets: {'ignore_subjects': ['jh101', 'jh102', 'jh103', 'jh104', 'jh105', 'jh106', 'jh107', 'jh108', 'la00', 'la01', 'la02', 'la03', 'la04', 'la05', 'la06', 'la07', 'la08', 'la09', 'la10', 'la11', 'la12', 'la13', 'la15', 'la16', 'la17', 'la20', 'la21', 'la22', 'la23', 'la24', 'la27', 'la28', 'la29', 'la31', 'nl01', 'nl02', 'nl03', 'nl04', 'nl05', 'nl06', 'nl07', 'nl08', 'nl09', 'nl10', 'nl11', 'nl12', 'nl13', 'nl14', 'nl15', 'nl16', 'nl17', 'nl18', 'nl19', 'nl20', 'nl21', 'nl22', 'nl23', 'nl24', 'pt10', 'pt11', 'pt12', 'pt13', 'pt14', 'pt15', 'pt16', 'pt17', 'pt2', 'pt3', 'pt6', 'pt7', 'pt8', 'pt9', 'tvb1', 'tvb11', 'tvb12', 'tvb14', 'tvb17', 'tvb18', 'tvb19', 'tvb2', 'tvb23', 'tvb27', 'tvb28', 'tvb29', 'tvb5', 'tvb7', 'tvb8', 'umf001', 'umf002', 'umf003', 'umf004', 'umf005', 'umf006', 'ummc001', 'ummc002', 'ummc003', 'ummc004', 'ummc005', 'ummc006', 'ummc007', 'ummc008', 'ummc009']}
Analyzing interictal task.
Found ['01', '02', '03', '04'] runs for interictal task.
Analyz

ERROR:root:Cannot activate multiple GUI eventloops


(84, 300)
Saved figure to /Users/adam2392/Dropbox/epilepsy_bids/derivatives/interictal/figures/originalsampling/fragility/average/sub-pt1/sub-pt1_ses-presurgery_task-interictal_acq-ecog_run-01_desc-perturbmatrix_ieeg.pdf
